In [1]:
#import ppa
import ppa
import numpy as np
import matplotlib.pyplot as plt
from PTMCMCSampler.PTMCMCSampler import PTSampler
from chainconsumer import ChainConsumer
from num2tex import num2tex
import json
import scipy.stats as ss
import corner
import os
    
PSR_DICT_LIST = ppa.Load_All_Pulsar_Info()
PSR_NAME_LIST = list(PSR_DICT_LIST.keys())
PSR_DICT_LIST

Do not have mpi4py package.
Do not have acor package


{'J0437-4715': {'PSR': 'J0437-4715',
  'RAJ': '04:37:15.8961737',
  'DECJ': '-47:15:09.110714',
  'DTE_DM': 0.156,
  'PX': 6.37,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0437-4715_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0437-4715_20cm.txt'},
  'RM': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0437-4715_ionFR_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0437-4715_ionFR_20cm.txt'}},
 'J0613-0200': {'PSR': 'J0613-0200',
  'RAJ': '06:13:43.975901',
  'DECJ': '-02:00:47.23742',
  'DTE_DM': 1.024,
  'PX': 1.01,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0613-0200_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0613-0200_20cm.txt'},
  'RM': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0613-0200_ionFR_10cm.txt',
   '20cm': 

In [ ]:
#=====================================================#
#     M + EFAC + EQUAD                                #
#=====================================================#
k = 0
logw_all = []
Res = {}
for i,psrn in enumerate(PSR_NAME_LIST):  
    res = {}
    for subset in ["10cm","20cm"]:
        
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ],order=2,iono="subt",subset=subset,nfreqs_dict={"10cm":0 , "20cm":0})
        array = ppa.Array([psr])
        if array.NPSR != 0:
            _lnlike = array.Generate_Lnlike_Function(adm_signal='none')

            def lnlike(x):
                l10_EFAC , l10_EQUAD  = np.array(x)
                return _lnlike( [l10_EFAC] , [l10_EQUAD] , [-3.] , [0.] ,1., -22. , -8. )

            l10_EFAC_lp , l10_EFAC_sp = ppa.gen_uniform_lnprior(-2,2)
            l10_EQUAD_lp , l10_EQUAD_sp = ppa.gen_uniform_lnprior(-8,2)

            def lnprior( x ):
                l10_EFAC , l10_EQUAD  =  x

                LP_1 = l10_EFAC_lp(l10_EFAC)
                LP_2 = l10_EQUAD_lp(l10_EQUAD)

                return  np.sum([LP_1,LP_2])


            init = np.array([l10_EFAC_sp() , l10_EQUAD_sp()  ])
            

            outdir = "SPA_Chain/white_"+psr.PSR_NAME+"_"+subset
            if os.path.isdir(outdir):
                pass
            else:
                print(psrn,subset)
                sampler = PTSampler(len(init),lnlike,lnprior,cov = np.diag(np.ones(len(init)))*0.01,\
                                    resume=False,outDir=outdir,verbose=True)
                sampler.sample(init,100000)

            chain  = np.loadtxt("SPA_Chain/white_"+psrn+"_"+subset+"/chain_1.txt",skiprows=5000)


            med1 = np.median( chain[:,0])
            min1 = med1 - np.quantile( chain[:,0],0.16 )
            max1 = np.quantile( chain[:,0],0.84 ) - med1 


            med2 = np.median( chain[:,1])
            min2 = med2 - np.quantile( chain[:,1],0.16 )
            max2 = np.quantile( chain[:,1],0.84 ) - med2



            logVol = np.log( 4 * 10   )
            logz0 = lnlike([0,-7.  ])
            logBF_white = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0
            logw_all.append(logBF_white)
            
            
            # with only psi12
            
            
            #print( k+1,'&',psrn,"& $%.2f$"%np.log10(np.median(psr.DPA_ERR[0])),"& $%.2f$"%np.log10(np.std(psr.DPA[0])) ,"& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f} $ &  & "%(med1,max1,min1,med2,max2,min2),"& %.1f"%(logBF_white)+"\\\\" )
            
            k += 1

In [ ]:
print(logw_all)

In [9]:
#=====================================================#
#     M + EFAC + EQUAD + Sred + Gamma                 #
#=====================================================#

Res = {}
k = 0
for i,psrn in enumerate(PSR_NAME_LIST):  
    res = {}
    for subset in ["10cm","20cm"]:
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ],order=2,iono="subt",subset=subset,nfreqs_dict={"10cm":30 , "20cm":30})
        array = ppa.Array([psr])
        if array.NPSR != 0:
            _lnlike = array.Generate_Lnlike_Function(adm_signal='none')

            def lnlike(x):
                l10_EFAC , l10_EQUAD , l10_S0red  ,Gamma = np.array(x)
                return _lnlike( [l10_EFAC] , [l10_EQUAD] , [l10_S0red] , [Gamma] ,1, -22 , -8 )

            l10_EFAC_lp , l10_EFAC_sp = ppa.gen_uniform_lnprior(-2,2)
            l10_EQUAD_lp , l10_EQUAD_sp = ppa.gen_uniform_lnprior(-8,2)
            l10_S0red_lp , l10_S0red_sp = ppa.gen_uniform_lnprior(-8,2)
            Gamma_lp    , Gamma_sp = ppa.gen_uniform_lnprior(-8,2)
            
            def lnprior( x ):
                l10_EFAC , l10_EQUAD , l10_S0red , Gamma =  x

                LP_1 = l10_EFAC_lp(l10_EFAC)
                LP_2 = l10_EQUAD_lp(l10_EQUAD)
                LP_3 = l10_S0red_lp(l10_S0red)
                LP_4 = Gamma_lp(Gamma)

                return  np.sum([LP_1,LP_2,LP_3,LP_4])

            

            init = np.array([l10_EFAC_sp() , l10_EQUAD_sp() , l10_S0red_sp() , Gamma_sp() ])

            outdir = "SPA_Chain/red30_"+psr.PSR_NAME+"_"+subset
            if os.path.isdir(outdir):
                pass
            else:
                print(psrn,subset)
                sampler = PTSampler(len(init),lnlike,lnprior,cov = np.diag(np.ones(len(init)))*0.01,\
                                    resume=False,outDir=outdir,verbose=True)
                sampler.sample(init,100000)

            chain  = np.loadtxt("SPA_Chain/red30_"+psrn+"_"+subset+"/chain_1.txt",skiprows=5000)


            med1 = np.median( chain[:,0])
            min1 = med1 - np.quantile( chain[:,0],0.16 )
            max1 = np.quantile( chain[:,0],0.84 ) - med1 


            med2 = np.median( chain[:,1])
            min2 = med2 - np.quantile( chain[:,1],0.16 )
            max2 = np.quantile( chain[:,1],0.84 ) - med2


            med3 = np.median( chain[:,2])
            min3 = med3 - np.quantile( chain[:,2],0.16 )
            max3 = np.quantile( chain[:,2],0.84 ) - med3


            med4 = np.median( chain[:,3])
            min4 = med4 - np.quantile( chain[:,3],0.16 )
            max4 = np.quantile( chain[:,3],0.84 ) - med4



            res.update( { subset:(med1,med2,med3,med4)} )
            logVol = np.log( 4 * 10 * 10 * 10   )
            logz0 = lnlike([0,-7 , -7 , 0   ])
            logBF = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0
            
            corner.corner(chain[:,[0,1,2,3]],labels=[r"$\log_{10}$EF" , r"$\log_{10}$EQ" , r"$\log_{10}S_{\rm red}$",r"$\Gamma$"],show_titles=True);
            plt.suptitle(psrn+"  "+subset)
            plt.savefig("Figures/"+psrn+"_"+subset+".jpg")
            plt.close()
        
            # with only psi12
            
            
            print( k+1,'&',psrn,"& %.2f"%np.log10(np.median(psr.DPA_ERR[0])),"& %.2f"%np.log10(np.std(psr.DPA[0])) ,\
                  "& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f} $ & $ % .2f^{+%.2f}_{-%.2f}$ & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2,med3,max3,min3,med4,max4,min4),\
                    "& %.1f"%(logBF-logw_all[k])+"\\\\" )
                    #"& %.1f"%(logBF)+"\\\\" )
            
            k += 1
        Res.update({psrn:res})

1 & J0437-4715 & -2.62 & -1.61 & $0.67^{+0.03}_{-0.03}$  & $-1.79^{+0.02}_{-0.02} $ & $ -3.79^{+0.57}_{-0.72}$ & $-4.32^{+1.32}_{-1.58}$ & 31.6\\
2 & J0437-4715 & -2.40 & -1.53 & $-1.09^{+0.62}_{-0.61}$  & $-1.62^{+0.01}_{-0.01} $ & $ -2.11^{+0.06}_{-0.06}$ & $-1.20^{+0.31}_{-0.35}$ & 233.8\\
3 & J0613-0200 & -0.84 & -0.83 & $-0.20^{+0.05}_{-0.05}$  & $-1.02^{+0.05}_{-0.06} $ & $ -5.01^{+2.23}_{-2.08}$ & $-1.86^{+2.85}_{-3.39}$ & 0.1\\
4 & J0613-0200 & -1.67 & -1.32 & $-0.02^{+0.03}_{-0.03}$  & $-1.59^{+0.02}_{-0.03} $ & $ -1.97^{+0.07}_{-0.07}$ & $-1.08^{+0.28}_{-0.30}$ & 75.3\\
5 & J0614-3329 & -1.12 & -1.21 & $-0.13^{+0.06}_{-0.09}$  & $-2.92^{+1.35}_{-3.53} $ & $ -5.23^{+1.92}_{-1.85}$ & $-2.67^{+3.17}_{-3.49}$ & -0.4\\
6 & J0711-6830 & -0.82 & -0.78 & $-0.13^{+0.02}_{-0.02}$  & $-1.23^{+0.05}_{-0.05} $ & $ -5.55^{+2.04}_{-1.68}$ & $-1.97^{+2.81}_{-3.48}$ & -0.2\\
7 & J0711-6830 & -1.34 & -1.18 & $-0.18^{+0.02}_{-0.02}$  & $-1.51^{+0.02}_{-0.02} $ & $ -1.99^{+0.07}_{-0.08}$ & $-1.0

27 & J1713+0747 & -2.07 & -1.59 & $-0.21^{+0.09}_{-0.11}$  & $-1.74^{+0.03}_{-0.03} $ & $ -2.13^{+0.09}_{-0.11}$ & $-0.82^{+0.38}_{-0.43}$ & 18.3\\
28 & J1713+0747 & -2.27 & -0.56 & $1.14^{+0.05}_{-0.05}$  & $-0.73^{+0.02}_{-0.02} $ & $ -1.10^{+0.06}_{-0.07}$ & $-1.15^{+0.83}_{-0.58}$ & 35.3\\
29 & J1730-2304 & -1.78 & -1.50 & $-0.12^{+0.04}_{-0.04}$  & $-1.75^{+0.03}_{-0.03} $ & $ -5.85^{+2.00}_{-1.46}$ & $-1.66^{+2.53}_{-3.48}$ & 0.0\\
30 & J1730-2304 & -1.98 & -0.89 & $0.69^{+0.02}_{-0.02}$  & $-4.00^{+2.06}_{-2.73} $ & $ -1.98^{+0.09}_{-0.11}$ & $-0.68^{+0.47}_{-0.47}$ & 10.8\\
31 & J1732-5049 & -1.07 & -0.88 & $-0.02^{+0.05}_{-0.06}$  & $-1.35^{+0.11}_{-0.10} $ & $ -5.06^{+2.07}_{-1.98}$ & $-2.61^{+3.10}_{-3.37}$ & 2.9\\
32 & J1744-1134 & -2.15 & -1.77 & $-0.08^{+0.05}_{-0.06}$  & $-1.87^{+0.02}_{-0.02} $ & $ -5.90^{+1.85}_{-1.45}$ & $-1.72^{+2.45}_{-3.25}$ & -0.5\\
33 & J1744-1134 & -2.27 & -1.52 & $0.23^{+0.05}_{-0.05}$  & $-1.77^{+0.02}_{-0.02} $ & $ -1.94^{+0.05}_{-0.05}$ & $-

In [8]:
with open("ppa/Parfile/spa_results.json",'w') as f:
    spa_results = json.dump(Res,f,indent=2)

[43186.96113939382, 53080.06982558298, 17.138269869468957, 534.4618979843831, 1.7892725134680916, 27.542212541427148, 859.6428351679845, 179.42228484629894, 4192.334462569131, 25.85646453022366, 30256.659435992886, -0.7007853744626118, 1368.9438576920602, 0.7499633492653857, 4177.086562971398, 21.211299667725655, 155.81922514172652, 1.5895069201914112, 38.374015042469296, 235.0336297661811, 37.90917672847513, 508.2912809102663, 38.543505784281706, 27799.031845727375, 0.5988114832136944, 35058.70674264737, 2034.5423531363967, 1058613.891256619, 329.11492375008925, 6171.697870868986, 43.34442473866589, 1229.519222330905, 11331.625150330052, -0.3269551845269234, 16217.153499785833, 13.813880334529017, 13.225779514984367, 484.5911191680191, 3579.1196911862467, 29302.337195157084, 22.569554673881214, 2669.649683620335, 18741.72664315737, 3156.52556385922, 24.99516146082516, 2966.316586438296, 4.0272079816444375, 735.3702913933587]


# Old

In [2]:

#=====================================================#
#    Summarize                                        #
#=====================================================#
test = 0
for i,psrn in enumerate(PSR_NAME_LIST[:]):  
    try:
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ] ,subset="10cm",iono="subt",order=2,nfreqs=0)
        lnlike,lnprior,init_gen = psr.gen_spa_likelihood("10cm")
        
        chain1  = np.loadtxt("SPA_Chain/m1_"+psrn+"_10cm/chain_1.txt",skiprows=5000)




        med1 = np.median( chain1[:,0])
        min1 = med1 - np.quantile( chain1[:,0],0.16 )
        max1 = np.quantile( chain1[:,0],0.84 ) - med1 


        med2 = np.median( chain1[:,1])
        min2 = med2 - np.quantile( chain1[:,1],0.16 )
        max2 = np.quantile( chain1[:,1],0.84 ) - med2 

        
        logz0 = lnlike([0,-np.inf])
        #test += logz0

        logBF1 = np.log( 0.02 * 0.01 * 0.1 ) - np.log( np.mean(np.exp( -( chain1[:,-4]-chain1[:,-4].max() ) ) ) )  + chain1[:,-4].max() - logz0



        psr_2 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=2 )
        psr_1 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=1 )
        psr_0 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=0 )

        lnlike_2,lnprior,init_gen = psr_2.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        lnlike_1,lnprior,init_gen = psr_1.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        lnlike_0,lnprior,init_gen = psr_0.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        
        dlnlike10 = lnlike_1([0,-np.inf ]) - lnlike_0([0,-np.inf ])
        dlnlike21 =  lnlike_2([0,-np.inf ]) - lnlike_1([0,-np.inf ])



        # print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR)) ,
        #       "& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),
        #       "&%.1f&%.1f& %.1f"%(logBF1*np.log10(np.exp(1)),logBF2*np.log10(np.exp(1)),logBF*np.log10(np.exp(1)) )+"\\\\" )
        
        print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR)),"&%.2f"%np.log10(np.std(psr.DPA[0])) ,
            "  & $% .2f^{+%.2f}_{-%.2f}$& $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),
            "&${:.1f}$".format(num2tex(dlnlike10)) , "&${:.1f}$".format(num2tex(dlnlike21))  ,  "&${:.1f}$".format(num2tex(logBF1)) +"\\\\"  )
            #   "  &  %.1e  &  %.1e  &  %.1f "%(logBF1,logBF2 - logBF1,logBF3-logBF2 )+"\\\\" )
    except:
        pass

In [ ]:
for i,psrn in enumerate(PSR_NAME_LIST[:]): 
    try:
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ] ,subset="10cm",iono="subt",order=2)
        print(psr.NOBS)
    except:
        pass